# Chunks - BERT


# Imports & Installs

In [ ]:
!pip install pydot --quiet
!pip install gensim --quiet
!pip install tensorflow==2.15.0 --quiet
!pip install tf_keras==2.15.0 --quiet
!pip install tensorflow-datasets==4.8 --quiet
# !pip install tensorflow-text==2.15.0 --quiet
!pip install transformers==4.17 --quiet

ERROR: Operation cancelled by user
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.1 MB/s eta 0:00:00


In [ ]:
#@title Imports
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping
#import tensorflow_datasets as tfds
# import tensorflow_text as tf_text
import transformers

from transformers import logging
logging.set_verbosity_error()

import sklearn as sk
from sklearn.model_selection import train_test_split
import os

import nltk
from nltk.data import find
from nltk.tokenize import sent_tokenize
nltk.download('punkt_tab')

import matplotlib.pyplot as plt

import re

import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

from transformers import BertTokenizer, TFBertModel
# bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
# bert_model = TFBertModel.from_pretrained('bert-base-cased')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# bert_model = TFBertModel.from_pretrained('bert-base-uncased')

In [ ]:
# 4-window plot of loss and accuracy for two models, for comparison

def make_plot(axs,
              model_history1,
              model_history2,
              model_1_name='model 1',
              model_2_name='model 2',
              ):
    box = dict(facecolor='yellow', pad=5, alpha=0.2)

    for i, metric in enumerate(['loss', 'accuracy']):
        y_lim_lower1 = np.min(model_history1.history[metric])
        y_lim_lower2 = np.min(model_history2.history[metric])
        y_lim_lower = min(y_lim_lower1, y_lim_lower2) * 0.9

        y_lim_upper1 = np.max(model_history1.history[metric])
        y_lim_upper2 = np.max(model_history2.history[metric])
        y_lim_upper = max(y_lim_upper1, y_lim_upper2) * 1.1

        for j, model_history in enumerate([model_history1, model_history2]):
            model_name = [model_1_name, model_2_name][j]
            ax1 = axs[i, j]
            ax1.plot(model_history.history[metric])
            ax1.plot(model_history.history['val_%s' % metric])
            ax1.set_title('%s - %s' % (metric, model_name))
            ax1.set_ylabel(metric, bbox=box)
            ax1.set_ylim(y_lim_lower, y_lim_upper)

# Read and Prepare Data

In [ ]:
chunk_highlights = pd.read_csv('/content/drive/MyDrive/266 Final Project/Processed Data/chunk_highlights.csv')
chunk_highlights['highlight'] = chunk_highlights['highlight'].fillna('')
chunk_highlights['label'] = chunk_highlights['highlight'].apply(lambda x: 0 if x =='' else 1)
chunk_highlights.head()

,service_id,doc_text,highlight,label
0,158,Privacy Policy The Apple Privacy Policy was up...,,0
1,158,"In addition to this Privacy Policy, we provide...",,0
2,158,You can review this information before enablin...,,0
3,158,Your California Privacy Disclosures \nInformat...,,0
4,158,Apple and its affiliates may share this person...,They may also combine it with other informatio...,1


Rebalance using random sampling

**Note** POSSIBLY INCLUDE RANDOM SAMPLING WITH REPLACEMENT HERE? Avoid selection and training with the same data points.

In [ ]:
class1_df = chunk_highlights[chunk_highlights['label'] == 1]
class0_df = chunk_highlights[chunk_highlights['label'] == 0].sample(n = len(chunk_highlights[chunk_highlights['label'] == 1]))
balanced_df = pd.concat([class1_df, class0_df])
balanced_df['label'].value_counts()

,count
label,
1,5924
0,5924


Create Train and Test datasets.

**Note 1:** Toggle between "highlight" (actual highlight text) and "label" (0/1) based on the need of the experiment.

**Note 2:** POSSIBLY INCLUDE CROSS VALIDATION HERE? Train/evaluate on different subsets of data.

In [ ]:
# X = balanced_df['doc_text'].values
y = balanced_df['label'].values

# X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 1)
X_train, X_test, y_train, y_test = train_test_split(balanced_df[['doc_text','highlight']], y, test_size = 0.2, random_state = 1)

print(X_train.shape)
print(X_test.shape)

(9478, 2)
(2370, 2)


# BERT

BERT Tokenization **without** separator token

In [ ]:
MAX_SEQUENCE_LENGTH = 512

def tokenize_with_separator(X, y):

  X_examples = [f" {bert_tokenizer.sep_token} ".join(sent_tokenize(x)) for x in X]

  bert_input_tokenized = bert_tokenizer(X_examples,
                                        max_length=MAX_SEQUENCE_LENGTH,
                                        truncation=True,
                                        padding='max_length',
                                        return_tensors='tf')

  bert_inputs = [bert_input_tokenized.input_ids,
                      bert_input_tokenized.token_type_ids,
                      bert_input_tokenized.attention_mask]

  bert_labels = np.array(y)

  return bert_inputs, bert_labels

# def tokenize_with_separator(X_train, X_test, y_train, y_test):

#   train_examples_str = [f" {bert_tokenizer.sep_token} ".join(sent_tokenize(X)) for X in X_train]
#   test_examples_str = [f" {bert_tokenizer.sep_token} ".join(sent_tokenize(X)) for X in X_test]

#   bert_train_tokenized = bert_tokenizer(
#                 train_examples_str,
#                 max_length=MAX_SEQUENCE_LENGTH,
#                 truncation=True,
#                 padding='max_length',
#                 return_tensors='tf')
#   bert_train_inputs = [bert_train_tokenized.input_ids,
#                       bert_train_tokenized.token_type_ids,
#                       bert_train_tokenized.attention_mask]

#   bert_train_labels = np.array(y_train)

#   bert_test_tokenized = bert_tokenizer(
#                 test_examples_str,
#                 max_length=MAX_SEQUENCE_LENGTH,
#                 truncation=True,
#                 padding='max_length',
#                 return_tensors='tf')
#   bert_test_inputs = [bert_test_tokenized.input_ids,
#                       bert_test_tokenized.token_type_ids,
#                       bert_test_tokenized.attention_mask]

#   bert_test_labels = np.array(y_test)

#   return bert_train_inputs, bert_train_labels, bert_test_inputs, bert_test_labels

In [ ]:
bert_train_inputs, bert_train_labels = tokenize_with_separator(X_train['doc_text'], y_train)
bert_test_inputs, bert_test_labels = tokenize_with_separator(X_test['doc_text'], y_test)

In [ ]:
def create_bert_cls_model(
                          # bert_base_model,
                          max_sequence_length=MAX_SEQUENCE_LENGTH,
                          hidden_size = 100,
                          dropout=0.3,
                          learning_rate=0.00005):
    """
    Build a simple classification model with BERT. Use the CLS Token output for classification purposes.
    """

    ### YOUR CODE HERE

    bert_base_model = TFBertModel.from_pretrained('bert-base-uncased')

    # bert_base_model.trainable = True
    # Freeze the embedding layer
    for param in bert_base_model.bert.embeddings.weights:
        param._trainable = False

    # Freeze the first 6 encoder layers (0-5)
    for param in bert_base_model.bert.encoder.layer[0:6]:
        param._trainable = False

    input_ids = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int64, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}

    bert_out = bert_base_model(bert_inputs)

    #pooler_token = bert_out[1]
    cls_token = bert_out[0][:, 0, :]

    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(cls_token)

    hidden = tf.keras.layers.Dropout(dropout)(hidden)

    classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(hidden)

    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])

    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                                 metrics=['accuracy', keras.metrics.Recall()])

    ### END YOUR CODE

    return classification_model, bert_base_model

#### Attempt 1

In [ ]:
bert_classification_model = create_bert_cls_model(bert_model)
bert_classification_model_history = bert_classification_model.fit(
    bert_train_inputs,
    bert_train_labels,
    validation_data = (bert_test_inputs, bert_test_labels),
    batch_size = 8,
    epochs = 3
)

Epoch 1/3


1185/1185 [==============================] - 646s 512ms/step - loss: 0.4923 - accuracy: 0.7744 - val_loss: 0.4846 - val_accuracy: 0.7726
Epoch 2/3
1185/1185 [==============================] - 603s 509ms/step - loss: 0.3601 - accuracy: 0.8547 - val_loss: 0.5059 - val_accuracy: 0.7553
Epoch 3/3
1185/1185 [==============================] - 602s 508ms/step - loss: 0.2430 - accuracy: 0.9062 - val_loss: 0.5721 - val_accuracy: 0.7878


In [ ]:
fig, axs = plt.subplots(2, 2)
fig.subplots_adjust(left=0.2, wspace=0.6)
make_plot(axs,
          bert_classification_model_history,
          bert_classification_model_history,
          model_1_name='bert',
          model_2_name='bert')

fig.align_ylabels(axs[:, 1])
fig.set_size_inches(18.5, 10.5)
plt.show()

In [ ]:
preds = bert_classification_model.predict(bert_test_inputs)

bert_test_predictions = pd.DataFrame()
bert_test_predictions['doc_text'] = test_examples_str
bert_test_predictions['y_test'] = y_test
bert_test_predictions['y_pred'] = (preds > 0.5).astype(int)
bert_test_predictions.head()

#### Attempt 2

In [ ]:
tf.keras.backend.clear_session()

bert_classification_model = create_bert_cls_model()

early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

bert_classification_model_history = bert_classification_model.fit(
    bert_train_inputs,
    bert_train_labels,
    validation_data = (bert_test_inputs, bert_test_labels),
    # [bert_train_inputs[0][:80], bert_train_inputs[1][:80], bert_train_inputs[2][:80]],
    # bert_train_labels[0:80],
    # validation_data = ([bert_test_inputs[0][:20], bert_test_inputs[1][:20], bert_test_inputs[2][:20]], bert_test_labels[0:20]),
    callbacks=[early_stopping],
    batch_size = 8,
    epochs = 3
)

Epoch 1/3


1185/1185 [==============================] - 601s 474ms/step - loss: 0.6723 - accuracy: 0.5770 - val_loss: 0.6199 - val_accuracy: 0.6629
Epoch 2/3
1185/1185 [==============================] - 594s 501ms/step - loss: 0.5966 - accuracy: 0.6977 - val_loss: 0.6105 - val_accuracy: 0.6667
Epoch 3/3
1185/1185 [==============================] - 594s 502ms/step - loss: 0.4828 - accuracy: 0.7790 - val_loss: 0.6260 - val_accuracy: 0.7004


#### Attempt 3, including separator token between sentences within chunks

In [ ]:
tf.keras.backend.clear_session()

bert_classification_model = create_bert_cls_model()

# early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

bert_classification_model_history = bert_classification_model.fit(
    bert_train_inputs,
    bert_train_labels,
    validation_data = (bert_test_inputs, bert_test_labels),
    # [bert_train_inputs[0][:5120], bert_train_inputs[1][:5120], bert_train_inputs[2][:5120]],
    # bert_train_labels[0:5120],
    # validation_data = ([bert_test_inputs[0][:1280], bert_test_inputs[1][:1280], bert_test_inputs[2][:1280]], bert_test_labels[0:1280]),
    # callbacks=[early_stopping],
    batch_size = 8,
    epochs = 3
)

Epoch 1/3


1185/1185 [==============================] - 923s 753ms/step - loss: 0.6735 - accuracy: 0.5827 - recall: 0.6101 - val_loss: 0.6272 - val_accuracy: 0.6376 - val_recall: 0.5091
Epoch 2/3
1185/1185 [==============================] - 839s 708ms/step - loss: 0.5817 - accuracy: 0.6940 - recall: 0.7295 - val_loss: 0.5771 - val_accuracy: 0.7063 - val_recall: 0.6983
Epoch 3/3
1185/1185 [==============================] - 888s 750ms/step - loss: 0.4457 - accuracy: 0.7970 - recall: 0.8233 - val_loss: 0.5900 - val_accuracy: 0.7287 - val_recall: 0.8248


In [ ]:
preds = bert_classification_model.predict(bert_test_inputs)

bert_test_predictions = pd.DataFrame()
bert_test_predictions['doc_text'] = test_examples_str
bert_test_predictions['y_test'] = y_test
bert_test_predictions['y_pred'] = (preds > 0.5).astype(int)
bert_test_predictions.head()

75/75 [==============================] - 95s 1s/step


,doc_text,y_test,y_pred
0,USSeek has no access to or control over these ...,0,1
1,PayPal specifically disclaims all warranties w...,0,0
2,First Party NVIDIA Remembers your preferred re...,0,1
3,The following are saved\n\nThe email address a...,0,0
4,We may freely transfer or assign this Agreemen...,1,1


In [ ]:
# Write output to file
filepath = '/content/drive/MyDrive/266 Final Project/Prediction Data/bert_chunk_predictions.csv'
bert_test_predictions.to_csv(filepath, index = False)

# BERT with Global Classification & Sentence Contribution

In [ ]:
test = 'here is string one. here is string two'
bert_tokenizer(test)

{'input_ids': [101, 2182, 2003, 5164, 2028, 1012, 2182, 2003, 5164, 2048, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
test = ['here is string one. here is string two. here is string third.']

sample = [f" {bert_tokenizer.sep_token} ".join(sent_tokenize(X)) for X in test]
sample = bert_tokenizer(sample)
sample

{'input_ids': [[101, 2182, 2003, 5164, 2028, 1012, 102, 2182, 2003, 5164, 2048, 1012, 102, 2182, 2003, 5164, 2353, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

[CLS] token id: 101

[SEP] token id: 102

1. Use CLS to output embedding to classify the entire input as 0 or 1.
2. Extract token embeddings for each segment between [SEP] tokens
  * Try mean pooling, or try using the [SEP] token embedding itself.
  * Analyze attention weights, or calculate cosine similarity between segment embedding and [CLS] embedding

In [ ]:
tf.keras.backend.clear_session()

bert_classification_model, trained_bert_model = create_bert_cls_model()

# early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

bert_classification_model_history = bert_classification_model.fit(
    bert_train_inputs,
    bert_train_labels,
    validation_data = (bert_test_inputs, bert_test_labels),
    batch_size = 8,
    epochs = 3
)

Epoch 1/3


1185/1185 [==============================] - 921s 750ms/step - loss: 0.6681 - accuracy: 0.5912 - recall: 0.6196 - val_loss: 0.6302 - val_accuracy: 0.6468 - val_recall: 0.9116
Epoch 2/3
1185/1185 [==============================] - 884s 746ms/step - loss: 0.5781 - accuracy: 0.7016 - recall: 0.7353 - val_loss: 0.5725 - val_accuracy: 0.7076 - val_recall: 0.8347
Epoch 3/3
1185/1185 [==============================] - 885s 747ms/step - loss: 0.4580 - accuracy: 0.7935 - recall: 0.8112 - val_loss: 0.5538 - val_accuracy: 0.7304 - val_recall: 0.8107


In [ ]:
# Checkpoint the model and fine-tuned bert embeddings
bert_classification_model.save("/content/drive/MyDrive/266 Final Project/model_checkpoints/bert_chunk_classification_model/", save_format="tf")

In [ ]:
# Get predictions for test data
preds = bert_classification_model.predict(bert_test_inputs)

X_test['y_test'] = y_test
X_test['y_pred'] = (preds > 0.5).astype(int)
X_test.head()

75/75 [==============================] - 95s 1s/step


,doc_text,highlight,y_test,y_pred
15349,Sellers may also offer Custom Offers to Buyers...,,0,0
24752,See more at httpconsumercal.orgcaliforniaonlin...,,0,1
15597,"At Anonyome Labs, Inc.\nAnonyome, we, us, or o...",,0,0
23630,Does our site allow thirdparty behavioral trac...,,0,1
15123,We may freely transfer or assign this Agreemen...,If any provision of this Agreement is held by ...,1,1


In [ ]:
# Example to show how we can get the mean of attention weights per token, for a specific test sample
# outputs = trained_bert_model(bert_test_inputs[0][0:1], bert_test_inputs[1][0:1], bert_test_inputs[2][0:1], output_attentions=True)
# attentions = outputs.attentions
# tf.reduce_mean(attentions[-1][:, :, 0, :], axis=1)

In [ ]:
# Calculate the sentence contribution for each chunk -> mean sentence attention weights
def calculate_sentence_contribution_attention(bert_model, inputs, tokens, attention):

  outputs = bert_model(inputs, tokens, attention, output_attentions=True)
  attentions = outputs.attentions
  token_attention = tf.reduce_mean(attentions[-1][:, :, 0, :], axis=1)

  # Identify the [SEP] token indeces for each input and store the indeces for all words between [SEP] tokens
  sep_tokens = np.where(inputs == bert_tokenizer.sep_token_id)[1]
  sentence_indices = [(1, sep_tokens[0])]
  for i in range(len(sep_tokens)-1):
    sentence_indices.append((sep_tokens[i] + 1, sep_tokens[i+1]))

  # Using the indeces found above, sum the attention weights for all tokens between the indeces
  sentence_contributions = []
  for start, end in sentence_indices:
    sentence_attention = tf.reduce_sum(token_attention[:, start:end], axis=1)
    sentence_contributions.append(sentence_attention.numpy()[0])

  # Normalize the aggregated sentence attention weights, to compare the importance of each sentence
  total_attention = sum(sentence_contributions)
  normalized_contributions = [x / total_attention for x in sentence_contributions]
  return normalized_contributions

In [ ]:
# Calculate the sentence contribution for each chunk -> cosine similarity CLS token and sentence max sentence embeddings
def calculate_sentence_contribution_max(bert_model, inputs, tokens, attention):

  # Get embeddings for CLS token
  outputs = bert_model(inputs, tokens, attention)
  token_embeddings = outputs[0]
  cls = token_embeddings[:,0,:]
  cls = tf.nn.l2_normalize(cls,axis = -1)

  # Identify the [SEP] token indeces for each input and store the indeces for all words between [SEP] tokens
  sep_tokens = np.where(inputs == bert_tokenizer.sep_token_id)[1]
  sentence_indices = [(1, sep_tokens[0])]
  for i in range(len(sep_tokens)-1):
    sentence_indices.append((sep_tokens[i] + 1, sep_tokens[i+1]))

  # Using the indeces found above, iterate through sentences and get embeddings for each sentence using max pooling
  sentence_contributions = []
  for start, end in sentence_indices:
    sentence_embedding = tf.reduce_max(token_embeddings[:, start:end], axis=1)
    sentence_embedding = tf.nn.l2_normalize(sentence_embedding, axis=-1)
    sentence_contributions.append(sentence_embedding.numpy()[0])

  # Calculate cosine similarity between CLS embeddings and sentence embeddings
  cosine_similarities = [(tf.reduce_sum(tf.multiply(cls, x))).numpy() for x in sentence_contributions]
  return cosine_similarities

In [ ]:
# Iterate through test examples, and calculate the sentence contribution using the fine-tuned bert model
sentence_contributions = []

for i in range(bert_test_inputs[0].shape[0]):
  # contribution = calculate_sentence_contribution_attention(trained_bert_model, bert_test_inputs[0][i:i+1], bert_test_inputs[1][i:i+1], bert_test_inputs[2][i:i+1])
  contribution = calculate_sentence_contribution_max(trained_bert_model, bert_test_inputs[0][i:i+1], bert_test_inputs[1][i:i+1], bert_test_inputs[2][i:i+1])
  sentence_contributions.append(contribution)

In [ ]:
# Append the sentence contributions to the dataframe
X_test['pred_sentence_contributions'] = sentence_contributions
X_test['pred_highlight_id'] = X_test['pred_sentence_contributions'].apply(lambda x: np.argmax(x))
X_test.head()

,doc_text,highlight,y_test,y_pred,pred_sentence_contributions,pred_highlight_id,highlight_id,highlight_id_check
15349,Sellers may also offer Custom Offers to Buyers...,,0,0,"[0.7437487, 0.72735083, 0.79266346]",2,NA,False
24752,See more at httpconsumercal.orgcaliforniaonlin...,,0,1,"[0.41247588, 0.43132368, 0.5052986]",2,NA,False
15597,"At Anonyome Labs, Inc.\nAnonyome, we, us, or o...",,0,0,"[0.28069827, 0.32059965, 0.28082293]",1,NA,False
23630,Does our site allow thirdparty behavioral trac...,,0,1,"[0.38011762, 0.31859756, 0.32117262]",0,NA,False
15123,We may freely transfer or assign this Agreemen...,If any provision of this Agreement is held by ...,1,1,"[0.33169833, 0.32375777, 0.26130384]",0,2,True


In [ ]:
# Remove pred_highlight_id for any row that was not predicted as class 1
# X_test['pred_highlight_id'] = X_test.apply(lambda x: 'NA' if x['y_pred'] == 0 else x['pred_highlight_id'], axis = 1)

In [ ]:
# Write output to file
filepath = '/content/drive/MyDrive/266 Final Project/Prediction Data/bert_chunk_predictions_20241124.csv'
X_test.to_csv(filepath, index = False)

Evaluate sentence contribution performance

In [ ]:
highlight_ids = []
found = 0

for index, row in X_test.iterrows():
  if row['y_test'] == 1:
    text = sent_tokenize(row['doc_text'])
    id = 0
    for i in text:
      if re.search(row['highlight'],i):
        found += 1
        break
      else:
        id += 1
    highlight_ids.append(id)
  else:
    highlight_ids.append('NA')

X_test['highlight_id'] = highlight_ids

In [ ]:
len(X_test)

2370

In [ ]:
X_test['highlight_id_check'] = X_test.apply(lambda x: x['highlight_id'] == x['pred_highlight_id'], axis = 1)

In [ ]:
X_test[(X_test['y_test'] == 1) & (X_test['y_pred'] == 1)]['highlight_id_check'].value_counts()

,count
highlight_id_check,
False,648
True,333


In [ ]:
X_test[(X_test['y_test'] == 1) & (X_test['y_pred'] == 1)]

,doc_text,highlight,y_test,y_pred,pred_sentence_contributions,pred_highlight_id,highlight_id,highlight_id_check
15123,We may freely transfer or assign this Agreemen...,If any provision of this Agreement is held by ...,1,1,"[0.33169833, 0.32375777, 0.26130384]",0,2,False
24971,They may be used by those companies to build a...,They may be used by those companies to build a...,1,1,"[0.34867042, 0.37558123, 0.34698248]",1,0,False
25331,Nous vous invitons lire la politique de confi...,Cookies des Rseaux Sociaux et Vidos \n\nCes co...,1,1,"[0.28301448, 0.29759884, 0.31927997]",2,2,True
17297,protection of our legitimate interests and rig...,This right applies exclusively to personal dat...,1,1,"[0.41333544, 0.32074556, 0.39448887]",0,2,False
3147,"We use cookies, pixels, local storage, and sim...","We use cookies, pixels, local storage, and sim...",1,1,"[0.30203, 0.34225902, 0.34603488]",2,0,False
...,...,...,...,...,...,...,...,...
19045,You will be responsible for preserving the con...,You will be responsible for preserving the con...,1,1,"[0.4263188, 0.418595, 0.4136476]",0,0,True
22354,Some of these providers allow you to opt out o...,Do Not Track Disclosure FamilyTreeDNA notes th...,1,1,"[0.34967083, 0.35321158, 0.43125176]",2,1,False
5657,You are responsible for your own actions You a...,You are responsible for your own actions You a...,1,1,"[0.30790967, 0.5327423, 0.39025262]",1,0,False
2629,"Because we do not control User Content,\nyou a...",You agree that Company will not be responsible...,1,1,"[0.35398307, 0.43872344, 0.37714162]",1,1,True


In [ ]:
X_test['highlight_id'].value_counts()

,count
highlight_id,
NA,1160
2,416
1,394
0,382
3,18


In [ ]:
X_test[(X_test['y_test'] == 1) & (X_test['y_pred'] == 1)]['pred_highlight_id'].value_counts()

,count
pred_highlight_id,
1,358
2,325
0,298


In [ ]:
X_test['doc_text'][24971]

'They may be used by those companies to build a profile of your interests and show you relevant advertising from College Board on other sites. They do not store directly personal information, but are based on uniquely identifying your browser and internet device. If you do not allow these cookies, you will experience less targeted advertising.Social Media Cookies Social Media Cookies These cookies are set by a range of social media services that we have added to the site to enable you to share our content with your friends and networks.'

In [ ]:
X_test['highlight'][24971]

'They may be used by those companies to build a profile of your interests and show you relevant advertising from College Board on other sites.'

In [ ]:
X_test['pred_highlight_id'][24971]

1

In [ ]:
X_test

IndexError: single positional indexer is out-of-bounds

# Appendix

## BERT with Cross Validation

In [ ]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
class1_df = chunk_highlights[chunk_highlights['label'] == 1][0:500]
class0_df = chunk_highlights[chunk_highlights['label'] == 0].sample(n = len(class1_df))
balanced_df = pd.concat([class1_df, class0_df])
balanced_df['label'].value_counts()

,count
label,
1,500
0,500


In [ ]:
X = balanced_df['doc_text'].values
y = balanced_df['label'].values
inputs = [x for x in X]

MAX_SEQUENCE_LENGTH = 256

bert_encodings = bert_tokenizer(inputs,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length',
              # return_tensors='tf',
              return_tensors='np')

input_ids = bert_encodings["input_ids"]
attention_masks = bert_encodings["attention_mask"]
token_type_ids = bert_encodings["token_type_ids"]

labels = np.array(y)

In [ ]:
k = 3
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
fold_no = 1
results = []

for train_index, val_index in skf.split(input_ids, labels):
  print(f"Training on Fold {fold_no}...")

  # train_index = tf.constant(train_index)
  # val_index = tf.constant(val_index)

  # Split data into train and validation sets
  X_train_ids, X_val_ids = input_ids[train_index], input_ids[val_index]
  X_train_masks, X_val_masks = attention_masks[train_index], attention_masks[val_index]
  X_train_token, X_val_token = token_type_ids[train_index], token_type_ids[val_index]
  y_train, y_val = labels[train_index], labels[val_index]

  bert_classification_model = create_bert_cls_model(bert_model)

  early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

  bert_classification_model_history = bert_classification_model.fit(
      [X_train_ids, X_train_masks, X_train_token],
      y_train,
      validation_data = ([X_val_ids, X_val_masks, X_val_token], y_val),
      callbacks=[early_stopping],
      batch_size = 8,
      epochs = 3
  )

  fold_no += 1

Training on Fold 1...
Epoch 1/3


84/84 [==============================] - 93s 617ms/step - loss: 0.7184 - accuracy: 0.5556 - val_loss: 0.5203 - val_accuracy: 0.7904
Epoch 2/3
84/84 [==============================] - 48s 569ms/step - loss: 0.4854 - accuracy: 0.7748 - val_loss: 0.4244 - val_accuracy: 0.7814
Epoch 3/3
84/84 [==============================] - 48s 568ms/step - loss: 0.3025 - accuracy: 0.8799 - val_loss: 0.5263 - val_accuracy: 0.7515
Training on Fold 2...
Epoch 1/3


84/84 [==============================] - 96s 633ms/step - loss: 0.7275 - accuracy: 0.5487 - val_loss: 0.6047 - val_accuracy: 0.7297
Epoch 2/3
84/84 [==============================] - 48s 573ms/step - loss: 0.4555 - accuracy: 0.7886 - val_loss: 0.5419 - val_accuracy: 0.7778
Epoch 3/3
84/84 [==============================] - 48s 573ms/step - loss: 0.2965 - accuracy: 0.8816 - val_loss: 0.4793 - val_accuracy: 0.7988
Training on Fold 3...
Epoch 1/3


84/84 [==============================] - 96s 617ms/step - loss: 0.7439 - accuracy: 0.5127 - val_loss: 0.6892 - val_accuracy: 0.5165
Epoch 2/3
84/84 [==============================] - 48s 572ms/step - loss: 0.7254 - accuracy: 0.5097 - val_loss: 0.6865 - val_accuracy: 0.5165
Epoch 3/3
84/84 [==============================] - 44s 524ms/step - loss: 0.6716 - accuracy: 0.5682 - val_loss: 0.5242 - val_accuracy: 0.7237


In [ ]:
bert_classification_model_history.history

{'loss': [0.743912398815155, 0.7254176735877991, 0.6716045141220093],
 'accuracy': [0.5127436518669128, 0.5097451210021973, 0.5682159066200256],
 'val_loss': [0.6892349720001221, 0.6864781975746155, 0.5241714119911194],
 'val_accuracy': [0.5165165066719055, 0.5165165066719055, 0.7237237095832825]}

In [ ]:
y_train.sum()/y_train.shape[0]

0.5

In [ ]:
y_val.sum()/y_val.shape[0]

0.5

## BERT + CNN

In [ ]:
def create_bert_cnn_model(bert_base_model,
                          max_sequence_length,
                          num_filters,
                          kernel_sizes,
                          dense_layer_dims,
                          dropout=0.3,
                          learning_rate=0.0005):

    #ensures bert model is fully trainable
    bert_base_model.trainable = True

    input_ids = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}

    bert_out = bert_base_model(bert_inputs)

    pooler_token = bert_out[1]
    pooler_token = tf.expand_dims(pooler_token, axis=-1)
    # cls_token = bert_out[0][:, 0, :]
    # conv_input = bert_out['pooler_output']
    # conv_input = tf.expand_dims(conv_input, axis=1)

    # conv_layer = keras.layers.Conv1D(filters=num_filters, kernel_size=kernel_sizes, activation='relu')(pooler_token)
    # conv_layer = tf.keras.layers.GlobalMaxPooling1D()(conv_layer)

    conv_layers_for_all_kernel_sizes = []
    for kernel_size, filters in zip(kernel_sizes, num_filters):
        conv_layer = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu')(pooler_token)
        conv_layer = keras.layers.GlobalMaxPooling1D()(conv_layer)
        conv_layers_for_all_kernel_sizes.append(conv_layer)

    conv_output = keras.layers.concatenate(conv_layers_for_all_kernel_sizes, axis=1)
    hidden_layer = keras.layers.Dropout(rate=dropout)(conv_output)

    for dense_layer_dim in dense_layer_dims:
        hidden_layer = keras.layers.Dense(dense_layer_dim, activation='relu')(hidden_layer)

    # hidden_layer = tf.keras.layers.Dense(dense_layer_dims, activation='relu', name='hidden_layer')(conv_layer)
    # dropout = tf.keras.layers.Dropout(dropout)(hidden_layer)

    classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(hidden_layer)

    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])

    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                                 metrics='accuracy')

    return classification_model

In [ ]:
tf.keras.backend.clear_session()

from transformers import BertTokenizer, TFBertModel
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
bert_model = TFBertModel.from_pretrained('bert-base-cased')

In [ ]:
num_filters = [64] #[100, 100, 50, 25]
kernel_sizes = [3] #[3, 4, 5, 10]
dense_layer_dims = [100]

# bert_cnn_model = create_bert_cnn_model(bert_model, 256, num_filters, kernel_sizes, dense_layer_dims)
# bert_cnn_model_history = bert_cnn_model.fit(
#     bert_train_inputs,
#     bert_train_labels,
#     validation_data = (bert_test_inputs, bert_test_labels),
#     batch_size = 8,
#     epochs = 3
# )

bert_cnn_model = create_bert_cnn_model(bert_model, 256, num_filters, kernel_sizes, dense_layer_dims)
bert_cnn_model_history = bert_cnn_model.fit(
    [bert_train_inputs[0][:80], bert_train_inputs[1][:80], bert_train_inputs[2][:80]],
    bert_train_labels[0:80],
    validation_data = ([bert_test_inputs[0][:20], bert_test_inputs[1][:20], bert_test_inputs[2][:20]], bert_test_labels[0:20]),
    batch_size = 8,
    epochs = 3
)

Epoch 1/5
10/10 [==============================] - 43s 802ms/step - loss: 0.7019 - accuracy: 0.4625 - val_loss: 0.7124 - val_accuracy: 0.4000
Epoch 2/5
10/10 [==============================] - 5s 509ms/step - loss: 0.6983 - accuracy: 0.5000 - val_loss: 0.7217 - val_accuracy: 0.4000
Epoch 3/5
10/10 [==============================] - 5s 505ms/step - loss: 0.7002 - accuracy: 0.5250 - val_loss: 0.7416 - val_accuracy: 0.4000
Epoch 4/5
10/10 [==============================] - 5s 487ms/step - loss: 0.7043 - accuracy: 0.4625 - val_loss: 0.7196 - val_accuracy: 0.4000
Epoch 5/5
10/10 [==============================] - 5s 512ms/step - loss: 0.6967 - accuracy: 0.5125 - val_loss: 0.7206 - val_accuracy: 0.4000
